In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/urdu-sarcastic-tweets-dataset/urdu_sarcastic_dataset.csv


In [2]:
df_raw=pd.read_csv('/kaggle/input/urdu-sarcastic-tweets-dataset/urdu_sarcastic_dataset.csv')
df_raw.head()

,urdu_text,is_sarcastic,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,🤣😂😂 ہو لینے دے میری شادی فسادن ٹھیک ہے کوجی نہ...,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,چل مہمانوں میں کھانا سرو کر چڑیل چاچی نوں دسدی...,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,کامران خان آپکی دن بھریہ زمہ داری لگائی گئی اپ...,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,نہیں پائین 😎,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,`` مراد علی شاہ کے بھیس میں ڈی جی آئی ایس آئی...,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df_raw.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5', 'Unnamed: 6','Unnamed: 7'], axis=1, inplace=True)
df_raw.head()

,urdu_text,is_sarcastic
0,🤣😂😂 ہو لینے دے میری شادی فسادن ٹھیک ہے کوجی نہ...,1.0
1,چل مہمانوں میں کھانا سرو کر چڑیل چاچی نوں دسدی...,1.0
2,کامران خان آپکی دن بھریہ زمہ داری لگائی گئی اپ...,0.0
3,نہیں پائین 😎,0.0
4,`` مراد علی شاہ کے بھیس میں ڈی جی آئی ایس آئی...,1.0


In [4]:
df_raw.columns = df_raw.columns.str.replace(' ', '')
df_raw[["urdu_text"]].astype(str)

import re
EMOJI_PATTERN = re.compile(
    "["
    "\U0001F1E0-\U0001F1FF"  # flags (iOS)
    "\U0001F300-\U0001F5FF"  # symbols & pictographs
    "\U0001F600-\U0001F64F"  # emoticons
    "😀-😃"
    "\U0001F680-\U0001F6FF"  # transport & map symbols
    "\U0001F700-\U0001F77F"  # alchemical symbols
    "\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
    "\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
    "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
    "\U0001FA00-\U0001FA6F"  # Chess Symbols
    "\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
    "\U00002702-\U000027B0"  # Dingbats
    "\U000024C2-\U0001F251" 
    "]+"
)

df_raw["urdu_text"]=df_raw['urdu_text'].str.replace(EMOJI_PATTERN, '', regex=True)
df_raw.head()

,urdu_text,is_sarcastic
0,ہو لینے دے میری شادی فسادن ٹھیک ہے کوجی نہیں ...,1.0
1,چل مہمانوں میں کھانا سرو کر چڑیل چاچی نوں دسدی...,1.0
2,کامران خان آپکی دن بھریہ زمہ داری لگائی گئی اپ...,0.0
3,نہیں پائین,0.0
4,`` مراد علی شاہ کے بھیس میں ڈی جی آئی ایس آئی...,1.0


In [5]:
df_raw= df_raw[df_raw['urdu_text'].notna()]

In [6]:
from sklearn.model_selection import GridSearchCV

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB

from sklearn.tree import DecisionTreeClassifier


from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

In [7]:
pipeline = {
    'svm':{
     'model':SVC(gamma='auto'),
        'params':{
            'C':[1,9,15],
            'kernel':['rbf','linear','poly','sigmoid']
        }
    },
    'random_forest':{
       'model':RandomForestClassifier(),
        'params':{
            'n_estimators':[1,5,10]
        }
    },
     'logistic_regression':{
        'model':LogisticRegression(solver='liblinear',multi_class='auto'),
         'params':{
            'C':[1,5,10]
        }
    },
    'BernoulliNB':{
        'model':BernoulliNB(),
        'params':{
           'alpha':range(1,10)
     }
    },
    'MultinomialNB':{
        'model':MultinomialNB(),
        'params':{
            'alpha':range(1,10)
        }
    },
    'DecisionTree' :{
     'model' :DecisionTreeClassifier(),
        'params':{
            'criterion':['gini','log_loss','entropy']
            
        }
    }
}

In [ ]:
X = cv.fit_transform(df_raw.urdu_text.values.astype('U'))
y = df_raw.is_sarcastic
scores = []
for model_name, mp in pipeline.items():
    clf = GridSearchCV(mp['model'],mp['params'], cv=10, return_train_score=False)
    clf.fit(X, y)
    scores.append({
        'model':model_name,
        'best_score':clf.best_score_,
        'best_params':clf.best_params_
    })
score_frame = pd.DataFrame(scores, columns=scores[0].keys()) ## overall report
score_frame

print('Best performance model')
score_frame[score_frame['best_score']==score_frame['best_score'].max()]